In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Lactose_Intolerance/GSE136395'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"The effects of a novel oral nutritional supplement as compared to standard care on body composition, physical function and skeletal muscle mRNA expression in Dutch older adults with (or at risk of) undernutrition"
!Series_summary	"In a randomized controlled trial, 82 older adults (>65y) with (or at risk of) undernutrition (n=82) were randomly allocated to 12 weeks of supplementation with a novel supplement (586 kcal, 22 g protein of which 50% whey and 50% casein, 206 mg ursolic acid, 7 g free BCAAs, 11 µg vitamin D) or standard care (600 kcal, 24g protein of which 100% casein, 4 µg vitamin D). Body weight increased significantly in the 12 weeks, both in the intervention group (+1.6 ± 0.2 kg, p<.0001) and in the standard care group (+1.8 ± 0.2 kg, p<.0001). Gait speed during 4m and 400m tests improved over time in the intervention group, whereas the standard care showed no improvements (time*treatment effects 400m: p=0.038 and 4m: p=0.048). Gene se

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = True
trait_row = None  # Lactose_Intolerance key not found
age_row = 2  # Identified key for age
gender_row = 0  # Identified key for gender

def convert_age(value):
    try:
        return float(value.split(": ")[1])
    except (ValueError, IndexError):
        return None

def convert_gender(value):
    try:
        return 1 if value.split(": ")[1] == '0' else 0  # Female = 0, Male = 1
    except (ValueError, IndexError):
        return None

def convert_trait(value):
    # Placeholder logic for trait conversion
    try:
        trait_value = value.split(": ")[1]
        if trait_value.lower() in ['yes', 'positive']:
            return 1
        elif trait_value.lower() in ['no', 'negative']:
            return 0
        else:
            return None
    except (ValueError, IndexError):
        return None

from utils.preprocess import save_cohort_info, geo_select_clinical_features

save_cohort_info('GSE136395', './preprocessed/Lactose_Intolerance/cohort_info.json', is_gene_available, trait_row is not None)

if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Lactose_Intolerance', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Lactose_Intolerance/trait_data/GSE136395.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Lactose_Intolerance/cohort_info.json
